## Import packages

In [1]:
using Revise
using ReactiveMP, Rocket, GraphPPL, Distributions
using LinearAlgebra, Random
using BenchmarkTools

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1423


## Settings

In [2]:
# seed for reproducibility
rng = MersenneTwister(1234)

# model parameters
A = [0.6 0.1 0.2; 0.3 0.7 0.3; 0.1 0.2 0.5];    # Process transition
B = [0.8 0.25 0.1; 0.1 0.5 0.6; 0.1 0.25 0.3];  # Observation transition
z0 = [1.0, 0.0, 0.0]; # Initial state

# benchmark settings
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 60;

## Generate data

In [3]:
function onehot_vec(dist::Categorical, rng)
    k = ncategories(dist); # get the number of categories
    x = zeros(k);
    x[rand(rng, dist)] = 1.0;

    return x
end

function generate_data(rng, z0, A, B; nr_samples=10)

    z = Vector{Vector{Float64}}(undef, nr_samples) # one-hot encoded state
    y = Vector{Vector{Float64}}(undef, nr_samples) # one-hot encoded observation

    z_prev = z0;
    for t = 1:nr_samples
        z[t] = onehot_vec(Categorical(A*z_prev), rng)
        y[t] = onehot_vec(Categorical(B*z[t]), rng)
        z_prev = z[t]
    end
    
    return y
end;

## ReactiveMP.jl scale factor extension

In [4]:
@rule Transition(:out, Marginalisation) (m_in::Categorical, m_a::PointMass, meta::ScaleFactorMeta) = begin 
    message = @call_rule Transition(:out, Marginalisation) (m_in = m_in, m_a = m_a)
    scalefactor = 0.0
    return ScaledMessage(message, scalefactor)
end

@rule Transition(:out, Marginalisation) (m_in::ScaledMessage, m_a::PointMass, meta::ScaleFactorMeta) = begin 
    A = mean(m_a)
    message = @call_rule Transition(:out, Marginalisation) (m_in = m_in.message, m_a = m_a)
    scalefactor = m_in.scale 
    return ScaledMessage(message, scalefactor)
end

@rule Transition(:in, Marginalisation) (m_out::PointMass, m_a::PointMass, meta::ScaleFactorMeta) = begin 
    A = mean(m_a)
    message = Categorical((A' * probvec(m_out)) ./ sum(A' * probvec(m_out)))
    scalefactor = -log(sum(A' * probvec(m_out)))
    return ScaledMessage(message, scalefactor)
end

@rule Transition(:in, Marginalisation) (m_out::ScaledMessage, m_a::PointMass, meta::ScaleFactorMeta) = begin 
    A = mean(m_a)
    message = Categorical((A' * probvec(m_out.message)) ./ sum(A' * probvec(m_out.message)))
    scalefactor = m_out.scale - log(sum(A' * probvec(m_out.message)))

    return ScaledMessage(message, scalefactor)
end;

LoadError: UndefVarError: ScaleFactorMeta not defined

In [ ]:
#Product function for equality node
function ReactiveMP.prod(::ProdAnalytical, left::ScaledMessage{ <: Categorical }, right::ScaledMessage{ <:Categorical })
    mean_left = probvec(left.message)
    mean_right = probvec(right.message)

    message = prod(ProdAnalytical(),left.message,right.message)
    scalefactor = left.scale + right.scale - log(dot(mean_left, mean_right))

    return ScaledMessage(message,scalefactor)
end;

## Inference by ReactiveMP (scale factors)

In [ ]:
@model [ default_meta = ScaleFactorMeta() ] function model_hmm_scalefactor(A, B; nr_samples=10)
    #define variables
    z = randomvar(nr_samples)
    y = datavar(Vector{Float64},nr_samples)

    cA = constvar(A)
    cB = constvar(B)

    # define initial state
    z_init ~ Categorical([1/3, 1/3, 1/3]) 

    z_prev = z_init

    for n = 1:nr_samples
        z[n] ~ Transition(z_prev, cA) 
        y[n] ~ Transition(z[n], cB) 
        z_prev = z[n]
    end

    return z, y
end

In [ ]:
function inference_hmm_scalefactor(data, A, B; nr_samples=10)

    model, (z, y) = model_hmm_scalefactor(A, B; nr_samples=nr_samples, options = (limit_stack_depth = 500, ));

    z_mar = keep(Vector{Marginal})

    z_subscript = subscribe!(getmarginals(z), z_mar)

    bmark_scf = @benchmark update!($y, $data)

    unsubscribe!(z_subscript)

    return z_mar, bmark_scf
end

## Inference by ReactiveMP (Bethe free energy)

In [5]:
@model function model_hmm_bfe(A, B; nr_samples=10)
    #define variables
    z = randomvar(nr_samples)
    y = datavar(Vector{Float64}, nr_samples)

    cA = constvar(A)
    cB = constvar(B)
    
    # define initial state
    z_init ~ Categorical([1/3, 1/3, 1/3])

    z_prev = z_init

    for n = 1:nr_samples
        z[n] ~ Transition(z_prev, cA)
        y[n] ~ Transition(z[n], cB) 
        z_prev = z[n]
    end

    return z, y
end

model_hmm_bfe (generic function with 1 method)

In [6]:
function inference_hmm_bfe(data, A, B; nr_samples=10)
    
    model, (z, y) = model_hmm_bfe(A, B; nr_samples=nr_samples, options = (limit_stack_depth = 500, ));

    z_mar = keep(Vector{Marginal})
    FE = keep(Float64)

    z_subscript = subscribe!(getmarginals(z), z_mar)
    fe_sub = subscribe!(score(Float64,BetheFreeEnergy(), model), FE)

    bmark = @benchmark update!($y, $data)

    unsubscribe!((z_subscript, fe_sub))

    return z_mar, FE, bmark
end

inference_hmm_bfe (generic function with 1 method)

## Correctness check and performance comparison

#### Scale factors (N=10)

In [7]:
data = generate_data(rng, z0, A, B; nr_samples=10);

In [ ]:
zmarginals_sf, bmark_sf = inference_hmm_scalefactor(data, A, B; nr_samples=10);
println(-zmarginals_sf[end][end].data.scale)
bmark_sf

#### Bethe free energy (N=10)

In [8]:
zmarginals, bfe, bmark_bfe = inference_hmm_bfe(data, A, B; nr_samples=10)
println(-bfe[end])
bmark_bfe

-10.76379656357213


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   82.400 μs … 111.910 ms  ┊ GC (min … max):  0.00% … 99.79%
 Time  (median):     107.900 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   133.897 μs ±   1.517 ms  ┊ GC (mean ± σ):  15.98% ±  1.41%

   ▇▁    ▄▆█▇▃                                                   
  ▅██▆▇▇███████▆▅▅▄▄▃▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  82.4 μs          Histogram: frequency by time          248 μs <

 Memory estimate: 80.20 KiB, allocs estimate: 1465.

#### Scale factors (N=100)

In [ ]:
data = generate_data(rng, z0, A, B; nr_samples=100);

In [ ]:
zmarginals_sf, bmark_sf = inference_hmm_scalefactor(data, A, B; nr_samples=100);
println(-zmarginals_sf[end][end].data.scale)
bmark_sf

#### Bethe free energy (N=100)

In [ ]:
zmarginals, bfe, bmark_bfe = inference_hmm_bfe(data, A, B; nr_samples=100)
println(-bfe[end])
bmark_bfe

#### Scale factors (N=1000)

In [ ]:
data = generate_data(rng, z0, A, B; nr_samples=1000);

In [ ]:
zmarginals_sf, bmark_sf = inference_hmm_scalefactor(data, A, B; nr_samples=1000);
println(-zmarginals_sf[end][end].data.scale)
bmark_sf

#### Bethe free energy (N=1000)

In [ ]:
zmarginals, bfe, bmark_bfe = inference_hmm_bfe(data, A, B; nr_samples=1000)
println(-bfe[end])
bmark_bfe